In [2]:
!uv add youtube-transcript-api

⠼ jupyter-server-terminals==0.5.3                                               Resolved 156 packages in 4.52s
⠙ Preparing packages... (0/1)                                                   ⠋ Preparing packages... (0/0)                                                   
⠙ Preparing packages... (0/1)----------------     0 B/473.68 KiB        
⠙ Preparing packages... (0/1)---------------- 2.68 KiB/473.68 KiB       
⠙ Preparing packages... (0/1)---------------- 18.68 KiB/473.68 KiB      
⠙ Preparing packages... (0/1)---------------- 34.68 KiB/473.68 KiB      
⠙ Preparing packages... (0/1)---------------- 50.68 KiB/473.68 KiB      
⠙ Preparing packages... (0/1)---------------- 62.02 KiB/473.68 KiB      
⠙ Preparing packages... (0/1)---------------- 78.02 KiB/473.68 KiB      
⠙ Preparing packages... (0/1)---------------- 94.02 KiB/473.68 KiB      
⠙ Preparing packages... (0/1)---------------- 110.02 KiB/473.68 KiB     
⠙ Preparing packages... (0/1)---------------- 125.05 KiB/473.68 KiB    

In [15]:
import re
from openai import OpenAI
import requests
import json
from minsearch import Index, VectorSearch
import pickle 
from youtube_transcript_api import YouTubeTranscriptApi
from typing import Any, Dict, List, Optional

client = OpenAI()

In [6]:
#!wget https://github.com/alexeygrigorev/ai-bootcamp-codespace/raw/refs/heads/main/week1/ph1PxZIkz1o.bin


In [16]:


video_id = 'ph1PxZIkz1o'

#ytt_api = YouTubeTranscriptApi()
#transcript = ytt_api.fetch(video_id)

with open(f'{video_id}.bin', 'rb') as f_in:
    transcript = pickle.load(f_in)



1407

In [17]:
def format_timestamp(seconds: float) -> str:
    """ Convert seconds to H:MM:SS if > 1 hour else MM:SS """
    total_seconds =  int(seconds)
    hours, remainder = divmod(total_seconds, 3600)
    minutes, secs = divmod(remainder, 60)

    if hours > 0:
        return f"{hours}:{minutes:02}:{secs:02}"
    else:
        return f"{minutes}:{secs:02}"
    
def make_subtitles(transcript) -> str:
    lines = []

    for entry in transcript:
        ts = format_timestamp(entry.start)
        text = entry.text.replace('\n', ' ')
        lines.append(ts + ' ' + text)
    
    return '\n'.join(lines)

subtitles = make_subtitles(transcript)

In [13]:
print(subtitles[:500])

0:00 So hi everyone. Uh today we are going to
0:02 talk about our upcoming course. The
0:05 upcoming course is called machine
0:06 learning zoom camp. And um this is
0:10 already I put the link in the
0:12 description. So if you're watching um
0:14 this video in recording or you're
0:17 watching it live, you go here in the
0:19 description after under this video and
0:21 then you see a link course. uh click on
0:25 that link and this bring you will bring
0:27 you to
0:29 this website this GitHub


In [18]:
instructions = """
Summarize the transcript and describe the main purpose of the video
and the main ideas. 

Also output chapters with time. Use usual sentence case, not Title Case for the chapter.

Output format: 

<OUTPUT>
Summary

timestamp chapter 
timestamp chapter
...
timestamp chapter
</OUTPUT>
"""

In [1]:
def llm(
    user_prompt: str,
    *,
    client: OpenAI,
    instructions: Optional[str] = None,
    model: str = "gpt-4o-mini",
) -> str:
    """
    Call the OpenAI Responses API with optional system instructions and return text.

    Args:
        user_prompt: The fully-assembled prompt to send as the user message.
        client: An initialized `OpenAI()` client.
        instructions: Optional system message to steer behavior (e.g., "answer only
            from context; say 'don't know' otherwise").
        model: OpenAI model name.

    Returns:
        The response text (assistant output) as a string.
    """
    messages = []
    if instructions:
        messages.append({"role": "system", "content": instructions})
    messages.append({"role": "user", "content": user_prompt})

    resp = client.responses.create(model=model, input=messages)
    return resp.output_text

NameError: name 'OpenAI' is not defined

In [19]:
answer = llm(subtitles, client=client, instructions=instructions)
print(answer)


NameError: name 'llm' is not defined

In [ ]:
def strip_matching_outer_html_tags(text: str) -> str:
    match = re.match(r"^\s*<(\w+)[^>]*>\s*(.*?)\s*</\1>\s*$", text, re.DOTALL)
    if match:
        return match.group(2).strip()
    return text.strip()

answer = strip_matching_outer_html_tags(answer)


In [25]:
from pydantic import BaseModel

class Chapter(BaseModel):
    timestamp: str
    title: str

class YTSummaryResponse(BaseModel):
    summary: str
    chapters: list[Chapter]


In [27]:
instructions = """
Summarize the transcript and describe the main purpose of the video
and the main ideas. 

Also output chapters with time. Use usual sentence case, not Title Case for the chapter.

More chapters is better than fewer chapters. Have a chapter at least every 3-5 minutes
""".strip()

messages = [
    {"role": "system", "content": instructions}, 
    {"role": "user", "content": subtitles}
]

response = client.responses.parse(
    model='gpt-4o-mini',
    input=messages,
    text_format=YTSummaryResponse
)


In [28]:
summary = response.output[0].content[0].parsed

print(summary.summary)
print()
for c in summary.chapters:
    print(c.timestamp, c.title)


The video discusses an upcoming course titled "Machine Learning Zoom Camp." The course aims to provide a structured learning path for individuals interested in machine learning engineering. It covers various topics including fundamental machine learning concepts, deployment techniques, and some computer vision basics. Participants can expect to gain relevant skills that enhance their job marketability as they learn to handle real-world ML engineering tasks. The session also includes a Q&A segment addressing prerequisites, course structure, and expected outcomes for participants.

0:00 Introduction to the course
2:34 Course content and updates
5:30 Job placement and prerequisites
10:14 Using programming and command line
15:38 Mathematical prerequisites explained
19:31 Course format and certificate details
25:23 Deadlines and project requirements
32:31 Interactivity and learning management
36:00 Course expectations and job readiness
40:30 Final thoughts and Q&A follow-up


In [13]:
def llm_structured(instructions, user_prompt, output_format, client, model="gpt-4o-mini"):
    messages = [
        {"role": "system", "content": instructions},
        {"role": "user", "content": user_prompt}
    ]

    response = client.responses.parse(
        model=model,
        input=messages,
        text_format=output_format
    )

    return response.output[0].content[0].parsed
    # or return response.output_parsed


In [14]:
summary = llm_structured(
    instructions=instructions,
    user_prompt=subtitles,
    client=client,
    output_format=YTSummaryResponse
)


NameError: name 'YTSummaryResponse' is not defined

In [32]:
summary

YTSummaryResponse(summary='The video is an overview and Q&A session regarding the upcoming "Machine Learning Zoom Camp" course, scheduled to start on September 15. The instructor discusses key aspects of the course, including its structure, prerequisites, and expected outcomes for participants. The course aims to teach foundational machine learning skills with a strong focus on engineering aspects, particularly deployment. Questions are answered regarding job prospects, prerequisites, and the depth of coverage for various topics like deep learning and computer vision. Overall, it serves as an informative session for prospective students to understand the course details and content better.', chapters=[Chapter(timestamp='0:00', title='Introduction to the upcoming course'), Chapter(timestamp='2:30', title='Course structure and updates'), Chapter(timestamp='5:10', title='Job placement prospects after the course'), Chapter(timestamp='8:10', title='Prerequisites for the course'), Chapter(tim

In [6]:
def sliding_window(seq, size, step):
    """Create overlapping chunks using sliding window approach."""
    if size <= 0 or step <= 0:
        raise ValueError("size and step must be positive")

    n = len(seq)
    result = []
    for i in range(0, n, step):
        batch = seq[i:i+size]
        result.append(batch)
        if i + size >= n:
            break

    return result

# Example:
print(sliding_window(list(range(18)), 10, 3))
# Output: [[0,1,2,3,4,5,6,7,8,9], [3,4,5,6,7,8,9,10,11,12], ...]


[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [3, 4, 5, 6, 7, 8, 9, 10, 11, 12], [6, 7, 8, 9, 10, 11, 12, 13, 14, 15], [9, 10, 11, 12, 13, 14, 15, 16, 17]]


In [8]:
def join_lines(transcript) -> str:
    """Join transcript entries into continuous text."""
    lines = []

    for entry in transcript:
        text = entry.text.replace('\n', ' ')
        lines.append(text)

    return ' '.join(lines)

def format_chunk(chunk):
    """Format a chunk with start/end timestamps and text."""
    time_start = format_timestamp(chunk[0].start)
    time_end = format_timestamp(chunk[-1].start)
    text = join_lines(chunk)

    return {
        'start': time_start,
        'end': time_end,
        'text': text
    }


In [9]:
chunks = []

# Experiment with different values: try (30, 10) for more granular chunks
for chunk in sliding_window(transcript, 60, 30):
    processed = format_chunk(chunk)
    chunks.append(processed)

print(f"Created {len(chunks)} chunks")


Created 46 chunks


In [11]:
from minsearch import Index

index = Index(text_fields=["text"])
index.fit(chunks)

# Test search:
results = index.search('Can I find a job after the course?', num_results=5)
print(results)


[{'start': '52:34', 'end': '55:07', 'text': "project I submitted was a fake course project. So there was nothing that's why I didn't get any points. Uh the reason I got uh nine uh is uh cuz I evaluated other peers. So that's why um like for each evalation I get three points. But this is how it's done. So the we evaluate projects by doing peer review and peer review is mandatory to complete the project. So if you submit a project but you don't do peer reviewing you fail the project and if you fail a project you fail the course. Right? So this very important to do peer reviews. Uh will the course make one job ready? Yes. If you put effort in the the the course and if you make a good project, if you also follow our recommendations to learn in public, this will definitely make you job ready. Uh what's the next path to follow after the completing the course? Uh to step into advanced stuff, find a job. That's the best way. Um cuz you can do courses forever, but I think you need to work on pr

In [25]:
import json

def search(query):
    """Search for relevant documents."""
    return index.search(
        query=query,
        num_results=15
    )

instructions = """
Answer the QUESTION based on the CONTEXT from the subtitles of a YouTube video.

Use only the facts from the CONTEXT when answering the QUESTION.

When answering the question, 
provide the citation in form of the video URL pointing at the timestamp where
this is discussed. If the question is discussed in multiple documents,
cite all of them.

Don't use markdown or any formatting in the output.
""".strip()

prompt_template = """
<VIDEO_ID>
{video_id}
</VIDEO_ID>

<QUESTION>
{question}
</QUESTION>

<CONTEXT>
{context}
</CONTEXT>
""".strip()

def build_prompt(question, search_results):
    context = json.dumps(search_results)
    return prompt_template.format(
        question=question,
        context=context,
        video_id=video_id
    ).strip()

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    response = llm(prompt, instructions=instructions)
    return response

# Test it:
answer = rag('Can I find a job after the course?')
print(answer)


Yes, you can find a job after the course. Many participants from previous editions of the course have successfully found jobs after graduating. The course is designed to equip you with essential skills for becoming job-ready in machine learning. However, it is important to put in the effort during the course and to engage in practical projects, as real experience is crucial for job readiness. Additionally, getting involved in volunteer opportunities can also help you build a portfolio. 

For more details, you can refer to the video at the following timestamps:
- Timestamp 1:21-3:49
- Timestamp 51:23-55:07
- Timestamp 56:18-58:55

Here is the link to the video: https://www.youtube.com/watch?v=ph1PxZIkz1o


In [24]:
def llm(user_prompt, instructions=None, model="gpt-4o-mini"):
    messages = []

    if instructions:
        messages.append({
            "role": "system",
            "content": instructions
        })

    messages.append({
        "role": "user",
        "content": user_prompt
    })

    response = client.responses.create(
        model=model,
        input=messages
    )

    return response.output_text

['I just discovered the course, can I still join?',
 'I just found out about this program. Can I still enroll?',
 'you can join the course any time.']

np.float32(0.99999976)

(948, 768)